In [1]:
import os, subprocess, re, traceback, sys
from glob import glob
import ants
import nibabel as nib
from nilearn import plotting
from nipype.interfaces import fsl
import matplotlib.pyplot as plt
import shutil
import pandas as pd
%matplotlib inline


# global settings
base_path = '/mnt/data_lab513/vqtran_data'
root_data = os.path.join(base_path, "data", "raw_data", "ADNI_NIfTI")
root_bias_correction = os.path.join(base_path, "data", "clean_data", "mri_bias_correction")
root_bet = os.path.join(base_path, "data", "clean_data", "mri_brain_extraction")
root_reg = os.path.join(base_path, "data", "clean_data", "mri_registration")
root_meta = os.path.join(base_path, "data", "meta_data")#, "Pre-Thesis_metadata", "ADNI") 
root_train = os.path.join(base_path, "data", "train_data")
#print(os.listdir(root_meta))

/home/vqtran/miniconda3/envs/test/lib/python3.6/site-packages/nilearn/datasets/__init__.py:90: FutureWarning:

Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.



In [6]:
ls

AD/  CN/  EMCI/  LMCI/


In [7]:
import pandas as pd


csv_table_dir = os.path.join(root_meta, "Pre-Thesis_5_21_2021_ADNI.csv")
df = pd.read_csv(csv_table_dir)

#df.head()
df.tail()

#print(df.to_string()) 

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
2085,I15948,002_S_0559,CN,M,79,1,MRI,MP-RAGE,Original,5/23/2006,DCM,NaN
2086,I240812,002_S_0413,CN,F,82,26,MRI,MPRAGE,Original,6/16/2011,DCM,11/22/2020
2087,I14437,002_S_0413,CN,F,76,1,MRI,MP-RAGE,Original,5/02/2006,DCM,2/22/2020
2088,I238627,002_S_0295,CN,M,90,26,MRI,MPRAGE,Original,6/02/2011,DCM,11/22/2020
2089,I13722,002_S_0295,CN,M,85,1,MRI,MP-RAGE,Original,4/18/2006,DCM,2/25/2020


In [8]:
#df.columns
#df["Subject"].nunique()
df["Subject"].to_list()

['941_S_4764',
 '941_S_4764',
 '941_S_4420',
 '941_S_4420',
 '941_S_4420',
 '941_S_4377',
 '941_S_4377',
 '941_S_4377',
 '941_S_4376',
 '941_S_4376',
 '941_S_4376',
 '941_S_4376',
 '941_S_4365',
 '941_S_4365',
 '941_S_4365',
 '941_S_4292',
 '941_S_4292',
 '941_S_4292',
 '941_S_4255',
 '941_S_4255',
 '941_S_4255',
 '941_S_4187',
 '941_S_4187',
 '941_S_4187',
 '941_S_4100',
 '941_S_4100',
 '941_S_4100',
 '941_S_4066',
 '941_S_4036',
 '941_S_4036',
 '941_S_4036',
 '941_S_2060',
 '941_S_1203',
 '941_S_1203',
 '941_S_1202',
 '941_S_1202',
 '941_S_1197',
 '941_S_1195',
 '941_S_1195',
 '941_S_1194',
 '153_S_4838',
 '153_S_4838',
 '153_S_4838',
 '153_S_4621',
 '153_S_4621',
 '153_S_4372',
 '153_S_4372',
 '153_S_4297',
 '153_S_4297',
 '153_S_4297',
 '153_S_4172',
 '153_S_4172',
 '153_S_4172',
 '153_S_4165',
 '153_S_4159',
 '153_S_4159',
 '153_S_4159',
 '153_S_4151',
 '153_S_4151',
 '153_S_4151',
 '153_S_4139',
 '153_S_4139',
 '153_S_4139',
 '153_S_4133',
 '153_S_4133',
 '153_S_4133',
 '153_S_41

In [9]:
#df['Image Data ID'].to_list()
len(df['Image Data ID'])

2090

In [10]:
os.listdir(root_reg)[:5]
#len(set(os.listdir(root_reg)))


['ADNI_002_S_0295_MR_MP-RAGE__br_raw_20060418193713091_1_S13408_I13722.nii.gz',
 'ADNI_002_S_0295_MR_MPRAGE_br_raw_20110602125255238_39_S110476_I238627.nii.gz',
 'ADNI_002_S_0413_MR_MP-RAGE__br_raw_20060502142705523_1_S13893_I14437.nii.gz',
 'ADNI_002_S_0413_MR_MPRAGE_br_raw_20110617102624650_40_S111992_I240812.nii.gz',
 'ADNI_002_S_0559_MR_MP-RAGE__br_raw_20060524075551726_1_S14875_I15948.nii.gz']

In [11]:
#get label from id_image
#Requirement: prevent data_leakage with id_subject
check_leakage_set = set()
check_leakage_list = list()
check_leakage_num = 0

for dirname, _, filenames in os.walk(root_reg):
    for filename in filenames:     
        #path = os.path.join(dirname, filename)
        id_image= filename[filename.index('_I')+1 : filename.index('.n')]
        #print(id_image)        
                
        id_subject = filename[5:15]
        #print(id_subject)
            
        if(id_subject in check_leakage_set):
            continue
        
        row_index = df.index[df['Image Data ID'] == id_image].tolist()
        label = df['Group'][row_index[0]]
        
        #print(label)
        check_leakage_set.add(id_subject)
        check_leakage_list.append(id_subject)
        check_leakage_num += 1
        #print("----------------")
        
        
print(check_leakage_num)
print(len(check_leakage_set))
print(len(check_leakage_list))
        
        #https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values

1005
1005
1005


In [12]:
#Check unique item to prevent data leakage
#df['Subject'].nunique()
#df['Image Data ID'].nunique()

In [13]:
def get_label_of_image_file (filename):
    #path = os.path.join(dirname, filename)
    id_image= filename[filename.index('_I')+1 : filename.index('.n')]
    #print(id_image)        
                
    id_subject = filename[5:15]
    #print(id_subject)
        
    row_index = df.index[df['Image Data ID'] == id_image].tolist()
    label = df['Group'][row_index[0]]
    return label

In [14]:
sample_dir = os.path.join(root_reg, 'ADNI_002_S_0295_MR_MP-RAGE__br_raw_20060418193713091_1_S13408_I13722.nii.gz')
get_label_of_image_file(sample_dir)

'CN'

In [15]:
os.getcwd()

'/mnt/data_lab513/vqtran_data/data/train_data'

In [20]:
#CREATE DATA DIRECTORY FOR READING DATA TO MODEL
CN_path = os.path.join(root_train, "CN")
EMCI_path = os.path.join(root_train, "EMCI")
LMCI_path = os.path.join(root_train, "LMCI")
AD_path = os.path.join(root_train, "AD")

shutil.rmtree(CN_path)
shutil.rmtree(EMCI_path)
shutil.rmtree(LMCI_path)
shutil.rmtree(AD_path)


#os.mkdir(os.path.join(base_path, "data", "train_data"))
os.mkdir(CN_path)
os.mkdir(EMCI_path)
os.mkdir(LMCI_path)
os.mkdir(AD_path)


In [17]:
#os.chdir(root_train)

In [21]:
#CHECK NUMBER OF FILE IN ROOT TRAIN 

# count = 0
# for dirname, _, filenames in os.walk(root_train):
#     for filename in filenames:
#         count += 1
#         #file_dir = os.path.join(dirname, filename)

# print(count)

In [22]:
count = 0
for dirname, _, filenames in os.walk(root_reg):
    for filename in filenames:
        file_dir = os.path.join(dirname, filename)
        
        #print(get_label_of_image_file (filename)) 
        label_to_class = get_label_of_image_file (filename)
        if (label_to_class == "CN"):
            shutil.copy2(file_dir, CN_path)
        if (label_to_class == "LMCI"):
            shutil.copy2(file_dir, LMCI_path)
        if (label_to_class == "EMCI"):
            shutil.copy2(file_dir, EMCI_path)
        if (label_to_class == "AD"):
            shutil.copy2(file_dir, AD_path)
        
        count+=1
        if (count % 100) == 0:
            print(count)

print("copy_finished")
print(count)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
copy_finished


In [23]:
print(count)

2091
